<a href="https://colab.research.google.com/github/Yyyyyjkim/review_analysis/blob/master/4_textrank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!apt-get update
!apt install chromium-chromedriver
!pip install requests
!pip install bs4
!pip install selenium
!pip install konlpy
!pip install sklearn

In [0]:
#-*- coding: utf-8 -*-

In [0]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re
import pickle
from konlpy.tag import Okt
import sklearn as sk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

In [0]:
with open('/content/drive/My Drive/comment_analysis/data/crawling.pickle', 'rb') as f:
  crawling = pickle.load(f)
with open('/content/drive/My Drive/comment_analysis/data/nlp.pickle', 'rb') as f:
  nlp = pickle.load(f)
with open('/content/drive/My Drive/comment_analysis/data/graph.pickle', 'rb') as f:
  graph = pickle.load(f)

In [0]:
class textrank:
  def __init__(self,url,sorting,pagenum,unit='self'):
    self.crawling = crawling()
    self.text = self.crawling.get_text(url,sorting,pagenum)

    self.nlp = nlp(self.text)
    self.tidy = self.nlp.tidy
    self.morphs = self.nlp.get_morphs()
    self.pos = self.nlp.get_pos()
    self.noun = self.nlp.get_noun()

    self.graph = graph()
    if unit=='self':
      self.graph_sent = self.graph.graph_sent(self.morphs)
      self.graph_word, self.words = self.graph.graph_word(self.morphs)
    if unit=='pos':
      self.graph_sent = self.graph.graph_sent(self.pos)
      self.graph_word, self.words = self.graph.graph_word(self.pos)
    if unit=='noun':
      self.graph_sent = self.graph.graph_sent(self.noun)
      self.graph_word, self.words = self.graph.graph_word(self.noun)
    
  def get_rank(self,graph='self',corpus='self',d=0.85,tol=1e-5,len=10):
    if graph=='self':
      graph = self.graph_sent
    if corpus=='self':
      corpus = self.tidy
    colsum = np.sum(graph,axis=0)
    weight = pd.DataFrame(graph).apply(lambda x:x/colsum,axis=1)
    weight = weight.fillna(0)
    pro_vec = np.ones(graph.shape[0]).reshape(-1,1)

    bias = (1-d)*pro_vec
    diff = sum(pro_vec)
    while diff > tol:
      past_vec = pro_vec.copy()
      pro_vec = bias+d*np.dot(weight,pro_vec)
      diff = sum(abs(past_vec-pro_vec))
    
    rank = {id: rank for id, rank in enumerate(pro_vec)}
    keywords = sorted(rank, key=lambda x: rank[x],reverse=True) #value를 기준으로 정렬하고 key return 

    summary = []
    for id in keywords[:len]:
      summary.append(corpus[id])
    
    return summary

In [0]:
crawl = crawling()
text_rank =crawl.get_text("https://smartstore.naver.com/jainystore/products/2455969597","랭킹순",10)
# text_date =crawl.get_text("https://smartstore.naver.com/jainystore/products/2455969597","최신순",10)
# text_desc =crawl.get_text("https://smartstore.naver.com/jainystore/products/2455969597","평점 높은순",10)
# text_asc =crawl.get_text("https://smartstore.naver.com/jainystore/products/2455969597","평점 낮은순",10)

In [0]:
rank = textrank("https://smartstore.naver.com/jainystore/products/2455969597","랭킹순",1,"noun")

In [0]:
summary = rank.get_rank()

In [0]:
summary

In [0]:
keywords = rank.get_rank(graph=rank.graph_word,corpus=rank.words)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [0]:
keywords

['베개', '어깨', '사용', '정도', '자고', '솔솔', '생각', '구매', '처음', '후기']